In [1]:
import base64

In [2]:
import requests
import datetime
from urllib.parse import urlencode

In [3]:
client_id = '384b551d70e746e58c198c49749bec9e'
client_secret = 'ad998aa695a6486184a94fd9569472cc'

In [21]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
            Returns a base64 encoded string 
        """
        client_id = self.client_id
        client_secret = self.client_secret
        client_creds = f'{client_id}:{client_secret}'
             
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
            
        client_creds_b64 = base64.b64encode(client_creds.encode())    
        return client_creds_b64.decode()
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds_b64}'
        }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'

        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client")
        
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type="albums", version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
        
    def get_album(self, _id):
        return self.get_resource(_id, resource_type="albums")  
        
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type="artists")
    
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type="artists"):
        if query is None:
            raise Exception("A query is required")
            
        if isinstance(query, dict):
            query = " ".join([f"{key}:{value}" for key, value in query.items()])
        if operator is not None and operator_query is not None:
            if operator.lower() == 'or' or operator.lower() == 'not': 
                operator = operator.upper()
            if isinstance(operator_query, str):
                query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [22]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.perform_auth()

True

In [23]:
spotify.search({"track": "Time", "artist": "hans"}, search_type="track")

q=track%3ATime+artist%3Ahans&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ATime+artist%3Ahans&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0YC192cP3KPCRWx8zr8MfZ'},
       'href': 'https://api.spotify.com/v1/artists/0YC192cP3KPCRWx8zr8MfZ',
       'id': '0YC192cP3KPCRWx8zr8MfZ',
       'name': 'Hans Zimmer',
       'type': 'artist',
       'uri': 'spotify:artist:0YC192cP3KPCRWx8zr8MfZ'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
 

In [24]:
spotify.get_artist("1hCkSJcXREhrodeIHQdav8")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
 'followers': {'href': None, 'total': 401207},
 'genres': ['german soundtrack',
  'scorecore',
  'soundtrack',
  'video game music'],
 'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
 'id': '1hCkSJcXREhrodeIHQdav8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/3192cec3d51144c7a061848927f89983835c5480',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/d33c97f0483a37ecf2d8309f99b1abaa1e499fd9',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/553f6b2aa9cb60ac1353f8efbd9edf3542332e8f',
   'width': 160}],
 'name': 'Ramin Djawadi',
 'popularity': 76,
 'type': 'artist',
 'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}

In [25]:
spotify.get_album("41zMFsCjcGenYKVJYUXU2n")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
   'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
   'id': '1hCkSJcXREhrodeIHQdav8',
   'name': 'Ramin Djawadi',
   'type': 'artist',
   'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'copyright

In [32]:
spotify.search(query="Em busca da meta", operator="NOT", operator_query="Meta",search_type="track")

q=Em+busca+da+meta+NOT+Meta&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Em+busca+da+meta+NOT+Meta&type=track&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}